## В этом файле отображен ход работы над заданием 5_2
### Здесь отображены отдельные артефакты и фрагменты кода, послужившие основой для main.py 

In [1]:
url = "https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p=2&region=1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1&type=4"

In [2]:
headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36"
    }

In [3]:
from bs4 import BeautifulSoup
import httpx

content = None

async def fetch_page(url):
    
    global content
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers=headers)
        if response.status_code == 200:
            content = response.text
        else:
            print(f"Ошибка получения страницы: {response.status_code}")

await fetch_page(url)


In [4]:
soup = BeautifulSoup(content, 'html')
links = soup.find_all(class_= '_93444fe79c--link--eoxce')

In [5]:
links = list(set([link['href'] for link in links]))

In [62]:
async def download_page(client: httpx.AsyncClient, url: str, data:list):
    response = await client.get(url, headers=headers)
    if response.status_code == 200:
        data.append(response.text)
    else:
        print(f"Ошибка получения страницы: {response.status_code}")
    return content

async def download_pages(links, data):
    """
    Asynchronously downloads a specified number of images to a given directory.
    
    Args:
    - num_images: Number of images to download.
    - output_dir: Directory to save the downloaded images.
    """
    async with httpx.AsyncClient() as session:
        tasks = [
            download_page(session, link, data)
            for link in links
        ]
        await asyncio.gather(*tasks)

### Обычно ожидания 3 секунд хватает. Оптимальным вариантом будет подключение прокси

In [74]:
import asyncio

In [63]:
data = []
for pair in zip(links[::2], links[1::2]):
    await download_pages(pair, data)
    await asyncio.sleep(3)

Ошибка получения страницы: 429
Ошибка получения страницы: 429


CancelledError: 

In [34]:
def get_flat_info(soup: BeautifulSoup):
    offer_summary_info_group = soup.find('div', {'data-name': 'OfferSummaryInfoGroup'})
    apartment_info = {}
    items = offer_summary_info_group.find_all('div', {'data-name': 'OfferSummaryInfoItem'})
    for item in items:
        key = item.find('p', class_='a10a3f92e9--color_gray60_100--mYFjS').text.strip()
        value = item.find('p', class_='a10a3f92e9--color_black_100--Ephi7').text.strip()
        apartment_info[key] = value

    return apartment_info

In [43]:
def get_building_info(soup: BeautifulSoup):
    house_info_group = soup.find('div', {'class': 'a10a3f92e9--group--K5ZqN', 'class': 'a10a3f92e9--right--_9uBM'})
    house_info = {}
    items = house_info_group.find_all('div', {'data-name': 'OfferSummaryInfoItem'})
    for item in items:
        key_element = item.find('p', class_='a10a3f92e9--color_gray60_100--mYFjS')
        value_element = item.find('p', class_='a10a3f92e9--color_black_100--Ephi7')
        if key_element and value_element:
            key = key_element.text.strip()
            value = value_element.text.strip()
            house_info[key] = value
    
    return house_info

In [46]:
def get_goods(soup: BeautifulSoup):
    features_layout = soup.find('div', {'data-name': 'FeaturesLayout'})
    amenities_list = []
    items = features_layout.find_all('div', {'data-name': 'FeaturesItem'})
    for item in items:
        amenity = item.text.strip()
        amenities_list.append(amenity)

    return(amenities_list)

In [48]:
from data import FlatData

In [60]:
len(data)

11

In [68]:
ready_data = []
for link, d in zip(links[:9], data[:9]):
    soup = BeautifulSoup(d, 'html.parser')
    title = soup.find('h1', class_='a10a3f92e9--title--vlZwT').text
    price = soup.find('div', class_='a10a3f92e9--amount--ON6i1').find('span').text
    attributes = {}
    fact_items = soup.find_all('div', class_='a10a3f92e9--item--iWTsg')
    for item in fact_items:
        attribute_span = item.find('span', class_='a10a3f92e9--color_black_100--Ephi7')
        if attribute_span:
            attribute = attribute_span.text.strip()
            value_span = attribute_span.find_next_sibling('span')
            if value_span:
                value = value_span.text.strip()
            else:
                value = ''
            attributes[attribute] = value
    description = soup.find('div', class_='a10a3f92e9--layout--BaqYw').text.strip()
    flat_ifno = get_flat_info(soup)
    building_info = get_building_info(soup)
    goods = get_goods(soup)
    ready_data.append(FlatData(link=link, name=title, price=price, description=description, other_description=attributes, about_flat=flat_ifno, about_building=building_info, goods=goods))
    

In [69]:
ready_data

[FlatData(link='https://www.cian.ru/rent/flat/278216280/', name='5-комн. квартира, 240 м²', price='600\xa0000\xa0₽/мес.', description='Без комиссии для арендатора!  С коллегами работаем 50/50  Лот  38596. Предлагается потрясающая двухуровневая 5-ти комнатная квартира в ЖК "Континенталь". \nПланировка первого уровня: кухня-гостиная, 2 спальни, 2 санузла, гардеробная комната, лоджия.\nПланировка второго уровня: спальня, кабинет (спальня), гардеробная комната, санузел.\nЭксклюзивный дизайнерский ремонт, мебель и техника от ведущих производителей Италии. Потолки 3,20 м. Презентабельная входная группа в доме. Есть подземная и наземная парковка.\nОперативный показ!', other_description={'Оплата ЖКХ': 'включена (без счётчиков)', 'Залог': '600 000 ₽', 'Комиссии': 'нет', 'Предоплата': '1 месяц', 'Срок аренды': 'от года', 'Торг': 'возможен', 'Условия проживания': 'можно с детьми и животными'}, about_flat={'Общая площадь': '240', 'Жилая площадь': '155', 'Площадь кухни': '30', 'Высота потолков': '3

In [70]:
import json

In [73]:
json_data = [flat.dict() for flat in ready_data]

# Записываем JSON на диск
with open('flats_data.json', 'w', encoding="utf-8") as json_file:
    json.dump(json_data, json_file, indent=4, ensure_ascii=False)
print("Data has been written to flats_data.json")


Data has been written to flats_data.json
